In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/company-data/department.csv
/kaggle/input/company-data/employees.csv


In [2]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 3.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 kB 13.7 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845513 sha256=6dafb89e1c7f86999e5a1f65b7b06d0b3768a2fe95631b5d3f68c7966317a169
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7
Note: you may need to restart the kernel to use updated packages.


In [3]:
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder.appName('hands_on').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/29 15:28:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType

In [6]:
person_list = [("Berry","","Allen",1,"M"),
        ("Oliver","Queen","",2,"M"),
        ("Robert","","Williams",3,"M"),
        ("Tony","","Stark",4,"F"),
        ("Rajiv","Mary","Kumar",5,"F")]

In [7]:
schema = StructType([ \
        StructField("firstname",StringType(),True), \
        StructField("middlename",StringType(),True), \
        StructField("lastname",StringType(),True), \
        StructField("id", IntegerType(), True), \
        StructField("gender", StringType(), True)])
 

In [8]:
 df = spark.createDataFrame(data=person_list,schema=schema)
 
 df.show(truncate=False)
 


+---------+----------+--------+---+------+
|firstname|middlename|lastname|id |gender|
+---------+----------+--------+---+------+
|Berry    |          |Allen   |1  |M     |
|Oliver   |Queen     |        |2  |M     |
|Robert   |          |Williams|3  |M     |
|Tony     |          |Stark   |4  |F     |
|Rajiv    |Mary      |Kumar   |5  |F     |
+---------+----------+--------+---+------+



In [9]:
df.printSchema()

root
 |-- firstname: string (nullable = true)
 |-- middlename: string (nullable = true)
 |-- lastname: string (nullable = true)
 |-- id: integer (nullable = true)
 |-- gender: string (nullable = true)



In [10]:
deptdf= spark.read.option("header",True).csv("/kaggle/input/company-data/department.csv")
deptdf.printSchema()

root
 |-- DEPARTMENT_ID: string (nullable = true)
 |-- DEPARTMENT_NAME: string (nullable = true)
 |-- MANAGER_ID: string (nullable = true)
 |-- LOCATION_ID: string (nullable = true)



In [11]:
deptdf.show()

+-------------+--------------------+----------+-----------+
|DEPARTMENT_ID|     DEPARTMENT_NAME|MANAGER_ID|LOCATION_ID|
+-------------+--------------------+----------+-----------+
|           10|      Administration|       200|       1700|
|           20|           Marketing|       201|       1800|
|           30|          Purchasing|       114|       1700|
|           40|     Human Resources|       203|       2400|
|           50|            Shipping|       121|       1500|
|           60|                  IT|       103|       1400|
|           70|    Public Relations|       204|       2700|
|           80|               Sales|       145|       2500|
|           90|           Executive|       100|       1700|
|          100|             Finance|       108|       1700|
|          110|          Accounting|       205|       1700|
|          120|            Treasury|        - |       1700|
|          130|       Corporate Tax|        - |       1700|
|          140|  Control And Credit|    

In [12]:

empdf = spark.read.option("header",True).option("inferSchema",True).csv("/kaggle/input/company-data/employees.csv")
empdf.printSchema()

root
 |-- EMPLOYEE_ID: integer (nullable = true)
 |-- FIRST_NAME: string (nullable = true)
 |-- LAST_NAME: string (nullable = true)
 |-- EMAIL: string (nullable = true)
 |-- PHONE_NUMBER: string (nullable = true)
 |-- HIRE_DATE: string (nullable = true)
 |-- JOB_ID: string (nullable = true)
 |-- SALARY: integer (nullable = true)
 |-- COMMISSION_PCT: string (nullable = true)
 |-- MANAGER_ID: string (nullable = true)
 |-- DEPARTMENT_ID: integer (nullable = true)



In [13]:
empdf.show()

+-----------+----------+---------+--------+------------+---------+----------+------+--------------+----------+-------------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE|    JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|
+-----------+----------+---------+--------+------------+---------+----------+------+--------------+----------+-------------+
|        198|    Donald| OConnell|DOCONNEL|650.507.9833|21-JUN-07|  SH_CLERK|  2600|            - |       124|           50|
|        199|   Douglas|    Grant|  DGRANT|650.507.9844|13-JAN-08|  SH_CLERK|  2600|            - |       124|           50|
|        200|  Jennifer|   Whalen| JWHALEN|515.123.4444|17-SEP-03|   AD_ASST|  4400|            - |       101|           10|
|        201|   Michael|Hartstein|MHARTSTE|515.123.5555|17-FEB-04|    MK_MAN| 13000|            - |       100|           20|
|        202|       Pat|      Fay|    PFAY|603.123.6666|17-AUG-05|    MK_REP|  6000|            - |       201|           20|


In [14]:
empdf.select("EMPLOYEE_ID","FIRST_NAME").show()

+-----------+----------+
|EMPLOYEE_ID|FIRST_NAME|
+-----------+----------+
|        198|    Donald|
|        199|   Douglas|
|        200|  Jennifer|
|        201|   Michael|
|        202|       Pat|
|        203|     Susan|
|        204|   Hermann|
|        205|   Shelley|
|        206|   William|
|        100|    Steven|
|        101|     Neena|
|        102|       Lex|
|        103| Alexander|
|        104|     Bruce|
|        105|     David|
|        106|     Valli|
|        107|     Diana|
|        108|     Nancy|
|        109|    Daniel|
|        110|      John|
+-----------+----------+
only showing top 20 rows



In [15]:
from pyspark.sql.functions import col
empdf.select(col("EMPLOYEE_ID"),col("FIRST_NAME")).show()

+-----------+----------+
|EMPLOYEE_ID|FIRST_NAME|
+-----------+----------+
|        198|    Donald|
|        199|   Douglas|
|        200|  Jennifer|
|        201|   Michael|
|        202|       Pat|
|        203|     Susan|
|        204|   Hermann|
|        205|   Shelley|
|        206|   William|
|        100|    Steven|
|        101|     Neena|
|        102|       Lex|
|        103| Alexander|
|        104|     Bruce|
|        105|     David|
|        106|     Valli|
|        107|     Diana|
|        108|     Nancy|
|        109|    Daniel|
|        110|      John|
+-----------+----------+
only showing top 20 rows



In [16]:
empdf.select("EMPLOYEE_ID","FIRST_NAME","SALARY").withColumn("NEW_SALARY",col("SALARY")+1000).show()

+-----------+----------+------+----------+
|EMPLOYEE_ID|FIRST_NAME|SALARY|NEW_SALARY|
+-----------+----------+------+----------+
|        198|    Donald|  2600|      3600|
|        199|   Douglas|  2600|      3600|
|        200|  Jennifer|  4400|      5400|
|        201|   Michael| 13000|     14000|
|        202|       Pat|  6000|      7000|
|        203|     Susan|  6500|      7500|
|        204|   Hermann| 10000|     11000|
|        205|   Shelley| 12008|     13008|
|        206|   William|  8300|      9300|
|        100|    Steven| 24000|     25000|
|        101|     Neena| 17000|     18000|
|        102|       Lex| 17000|     18000|
|        103| Alexander|  9000|     10000|
|        104|     Bruce|  6000|      7000|
|        105|     David|  4800|      5800|
|        106|     Valli|  4800|      5800|
|        107|     Diana|  4200|      5200|
|        108|     Nancy| 12008|     13008|
|        109|    Daniel|  9000|     10000|
|        110|      John|  8200|      9200|
+----------

In [17]:
empdf.withColumn("SALARY",col("SALARY") - 1000).select("EMPLOYEE_ID","FIRST_NAME","SALARY").show()

+-----------+----------+------+
|EMPLOYEE_ID|FIRST_NAME|SALARY|
+-----------+----------+------+
|        198|    Donald|  1600|
|        199|   Douglas|  1600|
|        200|  Jennifer|  3400|
|        201|   Michael| 12000|
|        202|       Pat|  5000|
|        203|     Susan|  5500|
|        204|   Hermann|  9000|
|        205|   Shelley| 11008|
|        206|   William|  7300|
|        100|    Steven| 23000|
|        101|     Neena| 16000|
|        102|       Lex| 16000|
|        103| Alexander|  8000|
|        104|     Bruce|  5000|
|        105|     David|  3800|
|        106|     Valli|  3800|
|        107|     Diana|  3200|
|        108|     Nancy| 11008|
|        109|    Daniel|  8000|
|        110|      John|  7200|
+-----------+----------+------+
only showing top 20 rows



In [18]:
empdf.withColumnRenamed("SALARY","EMP_SALARY").show()

+-----------+----------+---------+--------+------------+---------+----------+----------+--------------+----------+-------------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE|    JOB_ID|EMP_SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|
+-----------+----------+---------+--------+------------+---------+----------+----------+--------------+----------+-------------+
|        198|    Donald| OConnell|DOCONNEL|650.507.9833|21-JUN-07|  SH_CLERK|      2600|            - |       124|           50|
|        199|   Douglas|    Grant|  DGRANT|650.507.9844|13-JAN-08|  SH_CLERK|      2600|            - |       124|           50|
|        200|  Jennifer|   Whalen| JWHALEN|515.123.4444|17-SEP-03|   AD_ASST|      4400|            - |       101|           10|
|        201|   Michael|Hartstein|MHARTSTE|515.123.5555|17-FEB-04|    MK_MAN|     13000|            - |       100|           20|
|        202|       Pat|      Fay|    PFAY|603.123.6666|17-AUG-05|    MK_REP|      6000|         

In [19]:
empdf.drop("COMMISSION_PCT").show()

+-----------+----------+---------+--------+------------+---------+----------+------+----------+-------------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE|    JOB_ID|SALARY|MANAGER_ID|DEPARTMENT_ID|
+-----------+----------+---------+--------+------------+---------+----------+------+----------+-------------+
|        198|    Donald| OConnell|DOCONNEL|650.507.9833|21-JUN-07|  SH_CLERK|  2600|       124|           50|
|        199|   Douglas|    Grant|  DGRANT|650.507.9844|13-JAN-08|  SH_CLERK|  2600|       124|           50|
|        200|  Jennifer|   Whalen| JWHALEN|515.123.4444|17-SEP-03|   AD_ASST|  4400|       101|           10|
|        201|   Michael|Hartstein|MHARTSTE|515.123.5555|17-FEB-04|    MK_MAN| 13000|       100|           20|
|        202|       Pat|      Fay|    PFAY|603.123.6666|17-AUG-05|    MK_REP|  6000|       201|           20|
|        203|     Susan|   Mavris| SMAVRIS|515.123.7777|07-JUN-02|    HR_REP|  6500|       101|           40|
|        2

In [20]:
empdf.filter(col("SALARY")<5000).show()

+-----------+----------+-----------+--------+------------+---------+--------+------+--------------+----------+-------------+
|EMPLOYEE_ID|FIRST_NAME|  LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE|  JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|
+-----------+----------+-----------+--------+------------+---------+--------+------+--------------+----------+-------------+
|        198|    Donald|   OConnell|DOCONNEL|650.507.9833|21-JUN-07|SH_CLERK|  2600|            - |       124|           50|
|        199|   Douglas|      Grant|  DGRANT|650.507.9844|13-JAN-08|SH_CLERK|  2600|            - |       124|           50|
|        200|  Jennifer|     Whalen| JWHALEN|515.123.4444|17-SEP-03| AD_ASST|  4400|            - |       101|           10|
|        105|     David|     Austin| DAUSTIN|590.423.4569|25-JUN-05| IT_PROG|  4800|            - |       103|           60|
|        106|     Valli|  Pataballa|VPATABAL|590.423.4560|05-FEB-06| IT_PROG|  4800|            - |       103|           60|


In [21]:
empdf.filter(col("SALARY") < 5000).select("EMPLOYEE_ID","FIRST_NAME","SALARY").show(10)

+-----------+----------+------+
|EMPLOYEE_ID|FIRST_NAME|SALARY|
+-----------+----------+------+
|        198|    Donald|  2600|
|        199|   Douglas|  2600|
|        200|  Jennifer|  4400|
|        105|     David|  4800|
|        106|     Valli|  4800|
|        107|     Diana|  4200|
|        115| Alexander|  3100|
|        116|    Shelli|  2900|
|        117|     Sigal|  2800|
|        118|       Guy|  2600|
+-----------+----------+------+
only showing top 10 rows



In [22]:
empdf.filter((col("DEPARTMENT_ID")==50) & (col("SALARY")<5000)).select("EMPLOYEE_ID","FIRST_NAME","SALARY","DEPARTMENT_ID").show(10)

+-----------+----------+------+-------------+
|EMPLOYEE_ID|FIRST_NAME|SALARY|DEPARTMENT_ID|
+-----------+----------+------+-------------+
|        198|    Donald|  2600|           50|
|        199|   Douglas|  2600|           50|
|        125|     Julia|  3200|           50|
|        126|     Irene|  2700|           50|
|        127|     James|  2400|           50|
|        128|    Steven|  2200|           50|
|        129|     Laura|  3300|           50|
|        130|     Mozhe|  2800|           50|
|        131|     James|  2500|           50|
|        132|        TJ|  2100|           50|
+-----------+----------+------+-------------+
only showing top 10 rows



In [23]:
 empdf.filter("DEPARTMENT_ID != 50").select("EMPLOYEE_ID","FIRST_NAME","SALARY","DEPARTMENT_ID").show(10)

+-----------+----------+------+-------------+
|EMPLOYEE_ID|FIRST_NAME|SALARY|DEPARTMENT_ID|
+-----------+----------+------+-------------+
|        200|  Jennifer|  4400|           10|
|        201|   Michael| 13000|           20|
|        202|       Pat|  6000|           20|
|        203|     Susan|  6500|           40|
|        204|   Hermann| 10000|           70|
|        205|   Shelley| 12008|          110|
|        206|   William|  8300|          110|
|        100|    Steven| 24000|           90|
|        101|     Neena| 17000|           90|
|        102|       Lex| 17000|           90|
+-----------+----------+------+-------------+
only showing top 10 rows



In [24]:
empdf.distinct().show()

+-----------+----------+---------+--------+------------+---------+----------+------+--------------+----------+-------------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE|    JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|
+-----------+----------+---------+--------+------------+---------+----------+------+--------------+----------+-------------+
|        120|   Matthew|    Weiss|  MWEISS|650.123.1234|18-JUL-04|    ST_MAN|  8000|            - |       100|           50|
|        118|       Guy|   Himuro| GHIMURO|515.127.4565|15-NOV-06|  PU_CLERK|  2600|            - |       114|           30|
|        110|      John|     Chen|   JCHEN|515.124.4269|28-SEP-05|FI_ACCOUNT|  8200|            - |       108|          100|
|        123|    Shanta|  Vollman|SVOLLMAN|650.123.4234|10-OCT-05|    ST_MAN|  6500|            - |       100|           50|
|        124|     Kevin|  Mourgos|KMOURGOS|650.123.5234|16-NOV-07|    ST_MAN|  5800|            - |       100|           50|


In [25]:
empdf.dropDuplicates().show(10)

+-----------+----------+---------+--------+------------+---------+----------+------+--------------+----------+-------------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE|    JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|
+-----------+----------+---------+--------+------------+---------+----------+------+--------------+----------+-------------+
|        120|   Matthew|    Weiss|  MWEISS|650.123.1234|18-JUL-04|    ST_MAN|  8000|            - |       100|           50|
|        118|       Guy|   Himuro| GHIMURO|515.127.4565|15-NOV-06|  PU_CLERK|  2600|            - |       114|           30|
|        110|      John|     Chen|   JCHEN|515.124.4269|28-SEP-05|FI_ACCOUNT|  8200|            - |       108|          100|
|        123|    Shanta|  Vollman|SVOLLMAN|650.123.4234|10-OCT-05|    ST_MAN|  6500|            - |       100|           50|
|        124|     Kevin|  Mourgos|KMOURGOS|650.123.5234|16-NOV-07|    ST_MAN|  5800|            - |       100|           50|


In [26]:
empdf.dropDuplicates(["DEPARTMENT_ID", "HIRE_DATE"]).show(10)

+-----------+----------+----------+--------+------------+---------+--------+------+--------------+----------+-------------+
|EMPLOYEE_ID|FIRST_NAME| LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE|  JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|
+-----------+----------+----------+--------+------------+---------+--------+------+--------------+----------+-------------+
|        200|  Jennifer|    Whalen| JWHALEN|515.123.4444|17-SEP-03| AD_ASST|  4400|            - |       101|           10|
|        202|       Pat|       Fay|    PFAY|603.123.6666|17-AUG-05|  MK_REP|  6000|            - |       201|           20|
|        201|   Michael| Hartstein|MHARTSTE|515.123.5555|17-FEB-04|  MK_MAN| 13000|            - |       100|           20|
|        114|       Den|  Raphaely|DRAPHEAL|515.127.4561|07-DEC-02|  PU_MAN| 11000|            - |       100|           30|
|        119|     Karen|Colmenares|KCOLMENA|515.127.4566|10-AUG-07|PU_CLERK|  2500|            - |       114|           30|
|       

In [27]:
from pyspark.sql.functions import *
empdf.count()

50

In [28]:
empdf.select(count("salary")).show()

+-------------+
|count(salary)|
+-------------+
|           50|
+-------------+



In [29]:
empdf.select(count("salary").alias("total_count")).show()

+-----------+
|total_count|
+-----------+
|         50|
+-----------+



In [30]:
empdf.select(max("salary").alias("max_salary")).show()

+----------+
|max_salary|
+----------+
|     24000|
+----------+



In [31]:
empdf.select(min("salary").alias("min_salary")).show()

+----------+
|min_salary|
+----------+
|      2100|
+----------+



In [32]:
empdf.select(avg("salary").alias("avg_salary")).show()

+----------+
|avg_salary|
+----------+
|   6182.32|
+----------+



In [33]:
empdf.select(sum("salary").alias("sum_salary")).show()

+----------+
|sum_salary|
+----------+
|    309116|
+----------+



In [34]:
empdf.select("EMPLOYEE_ID","FIRST_NAME","DEPARTMENT_ID","SALARY").orderBy(col("DEPARTMENT_ID").asc(),col("SALARY").desc()).show(10)

+-----------+----------+-------------+------+
|EMPLOYEE_ID|FIRST_NAME|DEPARTMENT_ID|SALARY|
+-----------+----------+-------------+------+
|        200|  Jennifer|           10|  4400|
|        201|   Michael|           20| 13000|
|        202|       Pat|           20|  6000|
|        114|       Den|           30| 11000|
|        115| Alexander|           30|  3100|
|        116|    Shelli|           30|  2900|
|        117|     Sigal|           30|  2800|
|        118|       Guy|           30|  2600|
|        119|     Karen|           30|  2500|
|        203|     Susan|           40|  6500|
+-----------+----------+-------------+------+
only showing top 10 rows



In [35]:
empdf.groupBy("DEPARTMENT_ID").sum("SALARY").show(10)

+-------------+-----------+
|DEPARTMENT_ID|sum(SALARY)|
+-------------+-----------+
|           20|      19000|
|           40|       6500|
|          100|      51608|
|           10|       4400|
|           50|      85600|
|           70|      10000|
|           90|      58000|
|           60|      28800|
|          110|      20308|
|           30|      24900|
+-------------+-----------+



In [36]:
empdf.groupBy("DEPARTMENT_ID").max("SALARY").show(10)

+-------------+-----------+
|DEPARTMENT_ID|max(SALARY)|
+-------------+-----------+
|           20|      13000|
|           40|       6500|
|          100|      12008|
|           10|       4400|
|           50|       8200|
|           70|      10000|
|           90|      24000|
|           60|       9000|
|          110|      12008|
|           30|      11000|
+-------------+-----------+



In [37]:
empdf.groupBy("DEPARTMENT_ID").min("SALARY").show(10)

+-------------+-----------+
|DEPARTMENT_ID|min(SALARY)|
+-------------+-----------+
|           20|       6000|
|           40|       6500|
|          100|       6900|
|           10|       4400|
|           50|       2100|
|           70|      10000|
|           90|      17000|
|           60|       4200|
|          110|       8300|
|           30|       2500|
+-------------+-----------+



In [38]:
empdf.groupBy("DEPARTMENT_ID","JOB_ID").sum("SALARY").show(10)

+-------------+--------+-----------+
|DEPARTMENT_ID|  JOB_ID|sum(SALARY)|
+-------------+--------+-----------+
|           90| AD_PRES|      24000|
|           30|  PU_MAN|      11000|
|           70|  PR_REP|      10000|
|           50|  ST_MAN|      36400|
|           40|  HR_REP|       6500|
|           60| IT_PROG|      28800|
|           10| AD_ASST|       4400|
|           30|PU_CLERK|      13900|
|           50|ST_CLERK|      44000|
|           20|  MK_REP|       6000|
+-------------+--------+-----------+
only showing top 10 rows



In [39]:
empdf.printSchema()

root
 |-- EMPLOYEE_ID: integer (nullable = true)
 |-- FIRST_NAME: string (nullable = true)
 |-- LAST_NAME: string (nullable = true)
 |-- EMAIL: string (nullable = true)
 |-- PHONE_NUMBER: string (nullable = true)
 |-- HIRE_DATE: string (nullable = true)
 |-- JOB_ID: string (nullable = true)
 |-- SALARY: integer (nullable = true)
 |-- COMMISSION_PCT: string (nullable = true)
 |-- MANAGER_ID: string (nullable = true)
 |-- DEPARTMENT_ID: integer (nullable = true)



In [40]:
empdf.groupBy("DEPARTMENT_ID","JOB_ID").sum("SALARY", "EMPLOYEE_ID").show(10)

+-------------+--------+-----------+----------------+
|DEPARTMENT_ID|  JOB_ID|sum(SALARY)|sum(EMPLOYEE_ID)|
+-------------+--------+-----------+----------------+
|           90| AD_PRES|      24000|             100|
|           30|  PU_MAN|      11000|             114|
|           70|  PR_REP|      10000|             204|
|           50|  ST_MAN|      36400|             610|
|           40|  HR_REP|       6500|             203|
|           60| IT_PROG|      28800|             525|
|           10| AD_ASST|       4400|             200|
|           30|PU_CLERK|      13900|             585|
|           50|ST_CLERK|      44000|            2120|
|           20|  MK_REP|       6000|             202|
+-------------+--------+-----------+----------------+
only showing top 10 rows



In [41]:
empdf.groupBy("DEPARTMENT_ID").agg(sum("SALARY").alias("SUM_SALARY") , max("SALARY").alias("MAX_SALARY"), min("SALARY").alias("MIN_SALARY") , avg("SALARY").alias("AVG_SALARY")).show()

+-------------+----------+----------+----------+------------------+
|DEPARTMENT_ID|SUM_SALARY|MAX_SALARY|MIN_SALARY|        AVG_SALARY|
+-------------+----------+----------+----------+------------------+
|           20|     19000|     13000|      6000|            9500.0|
|           40|      6500|      6500|      6500|            6500.0|
|          100|     51608|     12008|      6900| 8601.333333333334|
|           10|      4400|      4400|      4400|            4400.0|
|           50|     85600|      8200|      2100|3721.7391304347825|
|           70|     10000|     10000|     10000|           10000.0|
|           90|     58000|     24000|     17000|19333.333333333332|
|           60|     28800|      9000|      4200|            5760.0|
|          110|     20308|     12008|      8300|           10154.0|
|           30|     24900|     11000|      2500|            4150.0|
+-------------+----------+----------+----------+------------------+



In [42]:
empdf.groupBy("DEPARTMENT_ID").agg(sum("SALARY").alias("SUM_SALARY") , max("SALARY").alias("MAX_SALARY"), min("SALARY").alias("MIN_SALARY") , avg("SALARY").alias("AVG_SALARY")).where(col("MAX_SALARY") >= 10000).show()

+-------------+----------+----------+----------+------------------+
|DEPARTMENT_ID|SUM_SALARY|MAX_SALARY|MIN_SALARY|        AVG_SALARY|
+-------------+----------+----------+----------+------------------+
|           20|     19000|     13000|      6000|            9500.0|
|          100|     51608|     12008|      6900| 8601.333333333334|
|           70|     10000|     10000|     10000|           10000.0|
|           90|     58000|     24000|     17000|19333.333333333332|
|          110|     20308|     12008|      8300|           10154.0|
|           30|     24900|     11000|      2500|            4150.0|
+-------------+----------+----------+----------+------------------+



In [43]:
empdf.groupBy("DEPARTMENT_ID").agg(sum("SALARY").alias("SUM_SALARY") , max("SALARY").alias("MAX_SALARY"), min("SALARY").alias("MIN_SALARY") , avg("SALARY").alias("AVG_SALARY")).where(col("MAX_SALARY") >= 10000).show()

+-------------+----------+----------+----------+------------------+
|DEPARTMENT_ID|SUM_SALARY|MAX_SALARY|MIN_SALARY|        AVG_SALARY|
+-------------+----------+----------+----------+------------------+
|           20|     19000|     13000|      6000|            9500.0|
|          100|     51608|     12008|      6900| 8601.333333333334|
|           70|     10000|     10000|     10000|           10000.0|
|           90|     58000|     24000|     17000|19333.333333333332|
|          110|     20308|     12008|      8300|           10154.0|
|           30|     24900|     11000|      2500|            4150.0|
+-------------+----------+----------+----------+------------------+



In [44]:
df = empdf.withColumn("EMP_GRADE", when( col("SALARY") > 15000 , "A").when( (col("SALARY") >= 10000) & ( col("SALARY") < 15000), "B").otherwise("C"))

In [45]:
df.select("EMPLOYEE_ID", "SALARY", "EMP_GRADE").show(10)

+-----------+------+---------+
|EMPLOYEE_ID|SALARY|EMP_GRADE|
+-----------+------+---------+
|        198|  2600|        C|
|        199|  2600|        C|
|        200|  4400|        C|
|        201| 13000|        B|
|        202|  6000|        C|
|        203|  6500|        C|
|        204| 10000|        B|
|        205| 12008|        B|
|        206|  8300|        C|
|        100| 24000|        A|
+-----------+------+---------+
only showing top 10 rows



In [46]:
empdf.createOrReplaceTempView("employee")

In [47]:
spark.sql(" select * from employee limit 5").show()

+-----------+----------+---------+--------+------------+---------+--------+------+--------------+----------+-------------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE|  JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|
+-----------+----------+---------+--------+------------+---------+--------+------+--------------+----------+-------------+
|        198|    Donald| OConnell|DOCONNEL|650.507.9833|21-JUN-07|SH_CLERK|  2600|            - |       124|           50|
|        199|   Douglas|    Grant|  DGRANT|650.507.9844|13-JAN-08|SH_CLERK|  2600|            - |       124|           50|
|        200|  Jennifer|   Whalen| JWHALEN|515.123.4444|17-SEP-03| AD_ASST|  4400|            - |       101|           10|
|        201|   Michael|Hartstein|MHARTSTE|515.123.5555|17-FEB-04|  MK_MAN| 13000|            - |       100|           20|
|        202|       Pat|      Fay|    PFAY|603.123.6666|17-AUG-05|  MK_REP|  6000|            - |       201|           20|
+-----------+---

In [48]:
df = spark.sql(" select employee_id,salary from employee")
df.show(10)

+-----------+------+
|employee_id|salary|
+-----------+------+
|        198|  2600|
|        199|  2600|
|        200|  4400|
|        201| 13000|
|        202|  6000|
|        203|  6500|
|        204| 10000|
|        205| 12008|
|        206|  8300|
|        100| 24000|
+-----------+------+
only showing top 10 rows



In [49]:
spark.sql("select department_id, sum(salary) as sum_salary from employee group by department_id").show()

+-------------+----------+
|department_id|sum_salary|
+-------------+----------+
|           20|     19000|
|           40|      6500|
|          100|     51608|
|           10|      4400|
|           50|     85600|
|           70|     10000|
|           90|     58000|
|           60|     28800|
|          110|     20308|
|           30|     24900|
+-------------+----------+



In [50]:
 spark.sql("select employee_id, department_id, rank() over(partition by department_id order by salary desc) as rank_salary from employee").show()

+-----------+-------------+-----------+
|employee_id|department_id|rank_salary|
+-----------+-------------+-----------+
|        200|           10|          1|
|        201|           20|          1|
|        202|           20|          2|
|        114|           30|          1|
|        115|           30|          2|
|        116|           30|          3|
|        117|           30|          4|
|        118|           30|          5|
|        119|           30|          6|
|        203|           40|          1|
|        121|           50|          1|
|        120|           50|          2|
|        122|           50|          3|
|        123|           50|          4|
|        124|           50|          5|
|        137|           50|          6|
|        129|           50|          7|
|        133|           50|          7|
|        125|           50|          9|
|        138|           50|          9|
+-----------+-------------+-----------+
only showing top 20 rows



In [51]:
empdf.join(deptdf, empdf.DEPARTMENT_ID == deptdf.DEPARTMENT_ID, "inner").show()

+-----------+----------+---------+--------+------------+---------+----------+------+--------------+----------+-------------+-------------+----------------+----------+-----------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|   EMAIL|PHONE_NUMBER|HIRE_DATE|    JOB_ID|SALARY|COMMISSION_PCT|MANAGER_ID|DEPARTMENT_ID|DEPARTMENT_ID| DEPARTMENT_NAME|MANAGER_ID|LOCATION_ID|
+-----------+----------+---------+--------+------------+---------+----------+------+--------------+----------+-------------+-------------+----------------+----------+-----------+
|        198|    Donald| OConnell|DOCONNEL|650.507.9833|21-JUN-07|  SH_CLERK|  2600|            - |       124|           50|           50|        Shipping|       121|       1500|
|        199|   Douglas|    Grant|  DGRANT|650.507.9844|13-JAN-08|  SH_CLERK|  2600|            - |       124|           50|           50|        Shipping|       121|       1500|
|        200|  Jennifer|   Whalen| JWHALEN|515.123.4444|17-SEP-03|   AD_ASST|  4400|            - |      

In [52]:
 empdf.join(deptdf, empdf.DEPARTMENT_ID == deptdf.DEPARTMENT_ID, "left").select(empdf.EMPLOYEE_ID, empdf.DEPARTMENT_ID, deptdf.DEPARTMENT_NAME).show(10)

+-----------+-------------+----------------+
|EMPLOYEE_ID|DEPARTMENT_ID| DEPARTMENT_NAME|
+-----------+-------------+----------------+
|        198|           50|        Shipping|
|        199|           50|        Shipping|
|        200|           10|  Administration|
|        201|           20|       Marketing|
|        202|           20|       Marketing|
|        203|           40| Human Resources|
|        204|           70|Public Relations|
|        205|          110|      Accounting|
|        206|          110|      Accounting|
|        100|           90|       Executive|
+-----------+-------------+----------------+
only showing top 10 rows



In [53]:
deptdf.show()

+-------------+--------------------+----------+-----------+
|DEPARTMENT_ID|     DEPARTMENT_NAME|MANAGER_ID|LOCATION_ID|
+-------------+--------------------+----------+-----------+
|           10|      Administration|       200|       1700|
|           20|           Marketing|       201|       1800|
|           30|          Purchasing|       114|       1700|
|           40|     Human Resources|       203|       2400|
|           50|            Shipping|       121|       1500|
|           60|                  IT|       103|       1400|
|           70|    Public Relations|       204|       2700|
|           80|               Sales|       145|       2500|
|           90|           Executive|       100|       1700|
|          100|             Finance|       108|       1700|
|          110|          Accounting|       205|       1700|
|          120|            Treasury|        - |       1700|
|          130|       Corporate Tax|        - |       1700|
|          140|  Control And Credit|    

In [54]:
deptdf.filter(deptdf.MANAGER_ID != "-").show()

+-------------+--------------------+----------+-----------+
|DEPARTMENT_ID|     DEPARTMENT_NAME|MANAGER_ID|LOCATION_ID|
+-------------+--------------------+----------+-----------+
|           10|      Administration|       200|       1700|
|           20|           Marketing|       201|       1800|
|           30|          Purchasing|       114|       1700|
|           40|     Human Resources|       203|       2400|
|           50|            Shipping|       121|       1500|
|           60|                  IT|       103|       1400|
|           70|    Public Relations|       204|       2700|
|           80|               Sales|       145|       2500|
|           90|           Executive|       100|       1700|
|          100|             Finance|       108|       1700|
|          110|          Accounting|       205|       1700|
|          120|            Treasury|        - |       1700|
|          130|       Corporate Tax|        - |       1700|
|          140|  Control And Credit|    

In [55]:
empdf.alias("emp1").join(empdf.alias("emp2") , col("emp1.manager_id") == col("emp2.employee_id"), "inner").select(col("emp1.manager_id"), col("emp2.first_name"), col("emp2.last_name")).show(10)

+----------+----------+---------+
|manager_id|first_name|last_name|
+----------+----------+---------+
|       201|   Michael|Hartstein|
|       205|   Shelley|  Higgins|
|       100|    Steven|     King|
|       100|    Steven|     King|
|       100|    Steven|     King|
|       100|    Steven|     King|
|       100|    Steven|     King|
|       100|    Steven|     King|
|       100|    Steven|     King|
|       100|    Steven|     King|
+----------+----------+---------+
only showing top 10 rows



In [56]:
empdf.alias("emp1").join(empdf.alias("emp2") , col("emp1.manager_id") == col("emp2.employee_id"), "inner").select(col("emp1.employee_id"), col("emp2.first_name"), col("emp2.last_name")).show(10)

+-----------+----------+---------+
|employee_id|first_name|last_name|
+-----------+----------+---------+
|        202|   Michael|Hartstein|
|        206|   Shelley|  Higgins|
|        124|    Steven|     King|
|        123|    Steven|     King|
|        122|    Steven|     King|
|        121|    Steven|     King|
|        120|    Steven|     King|
|        114|    Steven|     King|
|        102|    Steven|     King|
|        101|    Steven|     King|
+-----------+----------+---------+
only showing top 10 rows



In [57]:
empdf.join(deptdf, (empdf.DEPARTMENT_ID == deptdf.DEPARTMENT_ID) & (deptdf.LOCATION_ID == 1700), "inner").select(empdf.EMPLOYEE_ID, empdf.DEPARTMENT_ID, deptdf.DEPARTMENT_ID, deptdf.DEPARTMENT_NAME).show()

+-----------+-------------+-------------+---------------+
|EMPLOYEE_ID|DEPARTMENT_ID|DEPARTMENT_ID|DEPARTMENT_NAME|
+-----------+-------------+-------------+---------------+
|        200|           10|           10| Administration|
|        119|           30|           30|     Purchasing|
|        118|           30|           30|     Purchasing|
|        117|           30|           30|     Purchasing|
|        116|           30|           30|     Purchasing|
|        115|           30|           30|     Purchasing|
|        114|           30|           30|     Purchasing|
|        102|           90|           90|      Executive|
|        101|           90|           90|      Executive|
|        100|           90|           90|      Executive|
|        113|          100|          100|        Finance|
|        112|          100|          100|        Finance|
|        111|          100|          100|        Finance|
|        110|          100|          100|        Finance|
|        109| 

In [58]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
location_data = [(1700, "INDIA"), (1800, "USA")]
schema = StructType([ StructField("LOCATION_ID",IntegerType(),True), StructField("LOCATION_NAME",StringType(),True) ])
locDf = spark.createDataFrame(data=location_data,schema=schema)
locDf.printSchema()

root
 |-- LOCATION_ID: integer (nullable = true)
 |-- LOCATION_NAME: string (nullable = true)



In [59]:
locDf.show()

+-----------+-------------+
|LOCATION_ID|LOCATION_NAME|
+-----------+-------------+
|       1700|        INDIA|
|       1800|          USA|
+-----------+-------------+



In [60]:
empdf.join(deptdf, (empdf.DEPARTMENT_ID == deptdf.DEPARTMENT_ID) & (deptdf.LOCATION_ID == 1700), "inner").join(locDf, deptdf.LOCATION_ID == locDf.LOCATION_ID, "inner").select(empdf.EMPLOYEE_ID, empdf.DEPARTMENT_ID, deptdf.DEPARTMENT_NAME, locDf.LOCATION_NAME).show()

+-----------+-------------+---------------+-------------+
|EMPLOYEE_ID|DEPARTMENT_ID|DEPARTMENT_NAME|LOCATION_NAME|
+-----------+-------------+---------------+-------------+
|        205|          110|     Accounting|        INDIA|
|        206|          110|     Accounting|        INDIA|
|        108|          100|        Finance|        INDIA|
|        109|          100|        Finance|        INDIA|
|        110|          100|        Finance|        INDIA|
|        111|          100|        Finance|        INDIA|
|        112|          100|        Finance|        INDIA|
|        113|          100|        Finance|        INDIA|
|        100|           90|      Executive|        INDIA|
|        101|           90|      Executive|        INDIA|
|        102|           90|      Executive|        INDIA|
|        114|           30|     Purchasing|        INDIA|
|        115|           30|     Purchasing|        INDIA|
|        116|           30|     Purchasing|        INDIA|
|        117| 

In [61]:
def upperCase(in_str):
     out_str = in_str.upper()
     return out_str

In [62]:
print(upperCase("hello"))

HELLO


UDF

In [63]:
upperCaseUDF = udf(lambda z : upperCase(z) , StringType())
empdf.select(col("EMPLOYEE_ID") , col("FIRST_NAME"), col("LAST_NAME"), upperCaseUDF(col("FIRST_NAME")), upperCaseUDF(col("LAST_NAME"))).show()

+-----------+----------+---------+--------------------+-------------------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|<lambda>(FIRST_NAME)|<lambda>(LAST_NAME)|
+-----------+----------+---------+--------------------+-------------------+
|        198|    Donald| OConnell|              DONALD|           OCONNELL|
|        199|   Douglas|    Grant|             DOUGLAS|              GRANT|
|        200|  Jennifer|   Whalen|            JENNIFER|             WHALEN|
|        201|   Michael|Hartstein|             MICHAEL|          HARTSTEIN|
|        202|       Pat|      Fay|                 PAT|                FAY|
|        203|     Susan|   Mavris|               SUSAN|             MAVRIS|
|        204|   Hermann|     Baer|             HERMANN|               BAER|
|        205|   Shelley|  Higgins|             SHELLEY|            HIGGINS|
|        206|   William|    Gietz|             WILLIAM|              GIETZ|
|        100|    Steven|     King|              STEVEN|               KING|
|        101

In [64]:
@udf(returnType=StringType())
def upperCaseNew(in_str):
    out_str = in_str.upper()
    return out_str

In [65]:
empdf.select(col("EMPLOYEE_ID") , col("FIRST_NAME"), col("LAST_NAME"), upperCaseNew(col("FIRST_NAME")), upperCaseNew(col("LAST_NAME"))).show()

+-----------+----------+---------+------------------------+-----------------------+
|EMPLOYEE_ID|FIRST_NAME|LAST_NAME|upperCaseNew(FIRST_NAME)|upperCaseNew(LAST_NAME)|
+-----------+----------+---------+------------------------+-----------------------+
|        198|    Donald| OConnell|                  DONALD|               OCONNELL|
|        199|   Douglas|    Grant|                 DOUGLAS|                  GRANT|
|        200|  Jennifer|   Whalen|                JENNIFER|                 WHALEN|
|        201|   Michael|Hartstein|                 MICHAEL|              HARTSTEIN|
|        202|       Pat|      Fay|                     PAT|                    FAY|
|        203|     Susan|   Mavris|                   SUSAN|                 MAVRIS|
|        204|   Hermann|     Baer|                 HERMANN|                   BAER|
|        205|   Shelley|  Higgins|                 SHELLEY|                HIGGINS|
|        206|   William|    Gietz|                 WILLIAM|                 

In [66]:
from pyspark.sql.window import Window
windowSpec = Window.partitionBy("DEPARTMENT_ID").orderBy("SALARY")
empdf.withColumn("salary_rank", rank().over(windowSpec)).select("DEPARTMENT_ID","SALARY","salary_rank").show(10)

+-------------+------+-----------+
|DEPARTMENT_ID|SALARY|salary_rank|
+-------------+------+-----------+
|           10|  4400|          1|
|           20|  6000|          1|
|           20| 13000|          2|
|           30|  2500|          1|
|           30|  2600|          2|
|           30|  2800|          3|
|           30|  2900|          4|
|           30|  3100|          5|
|           30| 11000|          6|
|           40|  6500|          1|
+-------------+------+-----------+
only showing top 10 rows



In [67]:
windowSpec = Window.partitionBy("DEPARTMENT_ID").orderBy(col("SALARY").desc())
empdf.withColumn("salary_rank", rank().over(windowSpec)).select("DEPARTMENT_ID","SALARY","salary_rank").show(10)

+-------------+------+-----------+
|DEPARTMENT_ID|SALARY|salary_rank|
+-------------+------+-----------+
|           10|  4400|          1|
|           20| 13000|          1|
|           20|  6000|          2|
|           30| 11000|          1|
|           30|  3100|          2|
|           30|  2900|          3|
|           30|  2800|          4|
|           30|  2600|          5|
|           30|  2500|          6|
|           40|  6500|          1|
+-------------+------+-----------+
only showing top 10 rows



In [68]:

empdf.withColumn("SUM", sum("SALARY").over(Window.partitionBy("DEPARTMENT_ID").orderBy(col("SALARY").desc()))).select("DEPARTMENT_ID","SALARY","SUM").show(10)

+-------------+------+-----+
|DEPARTMENT_ID|SALARY|  SUM|
+-------------+------+-----+
|           10|  4400| 4400|
|           20| 13000|13000|
|           20|  6000|19000|
|           30| 11000|11000|
|           30|  3100|14100|
|           30|  2900|17000|
|           30|  2800|19800|
|           30|  2600|22400|
|           30|  2500|24900|
|           40|  6500| 6500|
+-------------+------+-----+
only showing top 10 rows



In [69]:
windowSpec = Window.partitionBy("DEPARTMENT_ID")
empdf.withColumn("SUM", sum("SALARY").over(windowSpec)).select("DEPARTMENT_ID","SALARY","SUM").show(100)

+-------------+------+-----+
|DEPARTMENT_ID|SALARY|  SUM|
+-------------+------+-----+
|           10|  4400| 4400|
|           20| 13000|19000|
|           20|  6000|19000|
|           30| 11000|24900|
|           30|  3100|24900|
|           30|  2900|24900|
|           30|  2800|24900|
|           30|  2600|24900|
|           30|  2500|24900|
|           40|  6500| 6500|
|           50|  2600|85600|
|           50|  2600|85600|
|           50|  8000|85600|
|           50|  8200|85600|
|           50|  7900|85600|
|           50|  6500|85600|
|           50|  5800|85600|
|           50|  3200|85600|
|           50|  2700|85600|
|           50|  2400|85600|
|           50|  2200|85600|
|           50|  3300|85600|
|           50|  2800|85600|
|           50|  2500|85600|
|           50|  2100|85600|
|           50|  3300|85600|
|           50|  2900|85600|
|           50|  2400|85600|
|           50|  2200|85600|
|           50|  3600|85600|
|           50|  3200|85600|
|           50

In [70]:
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", 104857600)
empdf.join(broadcast(deptdf), empdf.DEPARTMENT_ID == deptdf.DEPARTMENT_ID, "inner").select(empdf.EMPLOYEE_ID, empdf.DEPARTMENT_ID, deptdf.DEPARTMENT_NAME).show(100)

+-----------+-------------+----------------+
|EMPLOYEE_ID|DEPARTMENT_ID| DEPARTMENT_NAME|
+-----------+-------------+----------------+
|        198|           50|        Shipping|
|        199|           50|        Shipping|
|        200|           10|  Administration|
|        201|           20|       Marketing|
|        202|           20|       Marketing|
|        203|           40| Human Resources|
|        204|           70|Public Relations|
|        205|          110|      Accounting|
|        206|          110|      Accounting|
|        100|           90|       Executive|
|        101|           90|       Executive|
|        102|           90|       Executive|
|        103|           60|              IT|
|        104|           60|              IT|
|        105|           60|              IT|
|        106|           60|              IT|
|        107|           60|              IT|
|        108|          100|         Finance|
|        109|          100|         Finance|
|        1